In [41]:
#We track the avatar movement with the camera to find the x,y,z coordinates
#Every three seconds we will sample the strength of the electric field at the x,y,z
#coordinate
#We will also use is_point_inside_bvh to tell if we are inside (true)
#or outside (false)
#If inside we reward proportional to the field strength
#If outside we punish proportional to the field strength

In [43]:
#Packages
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [45]:
#Surfaces
'''
# Parametrize a sphere
def sphere_r(u, v):
    return np.array([np.sin(v) * np.cos(u), np.sin(v) * np.sin(u), np.cos(v)])
'''

'''
# Ellipsoid
a,b,c = 3.0,1.5,1.0
def ellipsoid_r(u,v):
    return np.array([a*np.sin(v)*np.cos(u),b*np.sin(v)*np.sin(u),c*np.cos(v)])
'''

'''
#Tannery pear
a=2.0
sqrt_2 = np.sqrt(2)
def pear_r(u,v):
    return np.array([a*np.sin(u)*np.cos(u)*np.cos(v),a*np.sin(u)*np.cos(u)*np.sin(v),2*sqrt_2*a*np.sin(u)])
'''

#Torus
r,r_2 = 10.0,20.0
def torus_r(u,v):
    return np.array([(r_2+r*np.cos(v))*np.cos(u),(r_2+r*np.cos(v))*np.sin(u),r*np.sin(v)])


    

In [47]:
#must find ranges to close the surface
r_func=torus_r #MAKE SURE TO CHANGE IF NEW SURFACE AND CHANGE RANGES
u_range=(0, 2 * np.pi)
v_range=(0, 2 * np.pi)
Nu=50
Nv=50

In [49]:
#These functions give the ability to plot the surface, find the centroid as if the surface were filled in to be uniformly dense, 
#get a magnitude plot of the field from a slice parallel to the xy-plane and plot with the centroid, find if a point is
#inside or outside the surface, and measure the field strength based on whether a point is inside or outside the surface


# 1) Mesh builder that wraps the u-seam
def build_param_mesh_periodic(r_func, u_range, v_range, Nu, Nv):

    """
    Build a closed triangular mesh of the surface by wrapping u at the seam.

    Parameters:
    - r_func   : function(u, v) -> np.array([x, y, z])
    - u_range  : (u_min, u_max)
    - v_range  : (v_min, v_max)
    - Nu, Nv   : resolution in u and v

    Returns:
    - triangles: list of np.ndarray of shape (3,3) representing mesh triangles
    """
    
    umin, umax = u_range
    vmin, vmax = v_range
    us = np.linspace(umin, umax, Nu, endpoint=False)
    vs = np.linspace(vmin, vmax, Nv)
    verts = np.zeros((Nv, Nu, 3))
    for i, v in enumerate(vs):
        for j, u in enumerate(us):
            verts[i,j] = r_func(u, v)
    tris = []
    for i in range(Nv-1):
        for j in range(Nu):
            j2 = (j+1) % Nu
            v0 = verts[i  , j ]
            v1 = verts[i  , j2]
            v2 = verts[i+1, j ]
            v3 = verts[i+1, j2]
            tris.append(np.stack([v0, v1, v3]))
            tris.append(np.stack([v0, v3, v2]))
    return tris



# 2) Find centroid
def compute_volume_centroid(triangles, ref_point=None):
    """
    Compute the volume centroid and total volume of a closed, star-shaped mesh by
    decomposing it into tetrahedra (ref_point, v0, v1, v2), using ABSOLUTE volumes.

    Parameters:
    - triangles : list of np.ndarray shape (3,3), each a mesh triangle [v0, v1, v2]
    - ref_point : np.array([x, y, z]) interior reference point (default origin)

    Returns:
    - centroid  : np.array([x_c, y_c, z_c])
    - total_vol : float
    """
    if ref_point is None:
        ref_point = np.zeros(3)

    total_vol = 0.0
    centroid  = np.zeros(3)

    for v0, v1, v2 in triangles:
        # signed volume of tetrahedron
        vol_signed = np.dot(v0 - ref_point,
                            np.cross(v1 - ref_point,
                                     v2 - ref_point)) / 6.0
        # use absolute to avoid orientation issues
        vol = abs(vol_signed)

        # centroid of that tetrahedron
        tet_centroid = (ref_point + v0 + v1 + v2) / 4.0

        # accumulate
        centroid  += tet_centroid * vol
        total_vol += vol

    centroid /= total_vol
    return centroid, total_vol




#determine if point is inside surface or not by using ray casting and triangle algorithm
#each tiny quadrilateral which comprise the surface is sampled then split into a triangle
#bounding‐volume hierarchy to hide most triangles behind a few bounding boxes
#Möller–Trumbore determines if R(t)= orig + t*dir hits the triangle surface (is intersection between the edges
#Ray‐AABB intersection we do a super-cheap check to see if the ray intersect that node’s axis-aligned box
#if no overlap we skip the subtree
#Traversal and “odd‐even” when we call hits = traverse_bvh(bvh_root, orig=point, dir, dir_inv) we see if ray 
#ray misses root's box. Otherwise If leaf → test each triangle, count how many intersections
#If internal → recurse into left and right children, sum their counts.
#inside = (hits % 2) == 1 to see if crossing surface amount is odd or even as ray casting does

# 3) Ray–triangle intersection (Möller–Trumbore)
def ray_intersect_triangle(orig, dir, v0, v1, v2):
    eps = 1e-9
    e1 = v1 - v0
    e2 = v2 - v0
    h  = np.cross(dir, e2)
    a  = np.dot(e1, h)
    if abs(a) < eps:
        return False
    f = 1.0 / a
    s = orig - v0
    u = f * np.dot(s, h)
    if u < 0 or u > 1:
        return False
    q = np.cross(s, e1)
    v = f * np.dot(dir, q)
    if v < 0 or u + v > 1:
        return False
    t = f * np.dot(e2, q)
    return t > eps

# 4) Robust ray–AABB intersection (no divide-by-zero)
def ray_intersect_aabb(orig, dir, bbox_min, bbox_max):
    tmin, tmax = -np.inf, np.inf
    for i in range(3):
        di = dir[i]
        if abs(di) > 1e-9:
            inv = 1.0 / di
            t1 = (bbox_min[i] - orig[i]) * inv
            t2 = (bbox_max[i] - orig[i]) * inv
            tmin = max(tmin, min(t1, t2))
            tmax = min(tmax, max(t1, t2))
        else:
            # parallel: origin must lie within slab
            if orig[i] < bbox_min[i] or orig[i] > bbox_max[i]:
                return False
    return tmax >= max(tmin, 0.0)

# 5) BVH node (stores bbox_min / bbox_max)
class BVHNode:
    def __init__(self, triangles, max_triangles=8):
        # compute this node’s bounding box
        pts = np.concatenate(triangles)
        self.bbox_min = np.min(pts, axis=0)
        self.bbox_max = np.max(pts, axis=0)

        if len(triangles) <= max_triangles:
            # leaf
            self.triangles = triangles
            self.left = self.right = None
        else:
            # internal: split along longest axis
            self.triangles = None
            centroids = [tri.mean(axis=0) for tri in triangles]
            axis = np.argmax(self.bbox_max - self.bbox_min)
            order = np.argsort([c[axis] for c in centroids])
            mid = len(triangles) // 2
            left_tris  = [triangles[i] for i in order[:mid]]
            right_tris = [triangles[i] for i in order[mid:]]
            self.left  = BVHNode(left_tris,  max_triangles)
            self.right = BVHNode(right_tris, max_triangles)

# 6) Traverse BVH to count ray-triangle hits
def traverse_bvh(node, orig, dir):
    # cull by the node’s box
    if not ray_intersect_aabb(orig, dir, node.bbox_min, node.bbox_max):
        return 0
    count = 0
    if node.triangles is not None:
        # leaf: brute-force
        for v0, v1, v2 in node.triangles:
            if ray_intersect_triangle(orig, dir, v0, v1, v2):
                count += 1
    else:
        # internal: recurse
        count += traverse_bvh(node.left,  orig, dir)
        count += traverse_bvh(node.right, orig, dir)
    return count

# 7) Point-in-solid test using odd-even rule
def is_point_inside_bvh(bvh_root, point, dir_vec=None):
    orig = np.array(point, dtype=float)
    if dir_vec is None:
        dir_vec = np.array([1.0, 0.1234, 0.4321])
    dir = dir_vec / np.linalg.norm(dir_vec)
    hits = traverse_bvh(bvh_root, orig, dir)
    return (hits % 2) == 1

#--------------------------------------------------------------------


# 8) Anti-coulomb field for outside surface case
def compute_E_r2(R, r_func, u_range, v_range, Nu, Nv):
    """
    Compute the “anti‐Coulomb” field at point R from a parametrized closed surface.
    The integrand grows like +r^2 instead of decaying like 1/r^2.

    Parameters:
    - R         : np.array([x, y, z]) observation point
    - r_func    : function(u, v) -> np.array([x, y, z]) parametrization of the surface
    - u_range   : (u_min, u_max)
    - v_range   : (v_min, v_max)
    - Nu, Nv    : int, resolution in u and v
    - K         : overall constant (e.g. σ/(4πε₀))

    Returns:
    - E         : np.array([Ex, Ey, Ez]) the computed field vector
    """
    umin, umax = u_range
    vmin, vmax = v_range
    du = (umax - umin) / Nu
    dv = (vmax - vmin) / Nv

    E = np.zeros(3)
    delta = 1e-6
    K = 2.0
    exponent = 0.25

    for i in range(Nu):
        u = umin + (i + 0.5) * du
        for j in range(Nv):
            v = vmin + (j + 0.5) * dv
            r_uv = r_func(u, v)

            # estimate partial derivatives
            ru = (r_func(u + delta, v) - r_uv) / delta
            rv = (r_func(u, v + delta) - r_uv) / delta

            # area element
            dA = np.linalg.norm(np.cross(ru, rv)) * du * dv

            # vector from surface patch to R
            d = R - r_uv
            r_mag = np.linalg.norm(d)
            if r_mag == 0:
                continue

            # anti‐Coulomb kernel
            E += (d / r_mag) * (r_mag**exponent) * dA
 

    return K * E


# 9) Compute field based on inside or outside condition
#Inside just drawn right to centroid and rewarded 
#Outside requires full surface calculation and is for punishment
def compute_field(R):
    """
    Hybrid field:
     - Outside the surface → anti‐Coulomb kernel over the mesh
     - Inside the surface  → standard Coulomb from the centroid
    """
    K_2 = 3.0
    R = np.asarray(R, dtype=float)
    exponent = 1.0
    
    # inside vs. outside
    if is_point_inside_bvh(bvh, R):
        # standard Coulomb from the centroid
        d = R - vol_centroid
        r = np.linalg.norm(d)
        if r < 1e-12:
            return np.zeros(3)          # avoid singularity at the centroid
        else:
            E = (d / r) * (1 / r**exponent)
            return K_2 * E           
            
    else:
        # anti-Coulomb across the surface
        return compute_E_r2(R, r_func, u_range, v_range, Nu, Nv)

In [51]:
#Compute the solid‐body centroid and volume
tris = build_param_mesh_periodic(r_func, u_range, v_range, Nu, Nv)
vol_centroid = compute_volume_centroid(tris)[0] #ref point already defined previously
volume = compute_volume_centroid(tris)[1]
print("Centroid:", vol_centroid)
#print("Volume:", volume)

Centroid: [-1.37111282e-16 -1.57210550e-16 -5.08752971e-16]


In [53]:
# Test point:
if __name__ == "__main__":
    # Build mesh and BVH
    bvh = BVHNode(tris, max_triangles=8) #tris from before

    # Test points     True - inside closed mesh     False - outside closed mesh
    p1=np.array([15.0,8,3])
    p2=np.array([7.0,0,0])
    print(is_point_inside_bvh(bvh, p1))   
    print(is_point_inside_bvh(bvh, p2))  

True
False


In [55]:
#ε0 = 8.854187817e-12
#σ  = 1e-6      # surface‐charge density for anti‐Coulomb case
#K2 = σ/(4*np.pi*ε0)

#q_inside = σ * total_surface_area   # or some chosen total charge
#K1 = q_inside/(4*np.pi*ε0)




# Example usage:
points_example = [np.array(p1),np.array(p2)]

for P in points_example:
    E = compute_field(P)
    E_mag = np.linalg.norm(E)
    print(f"P={P}, E={E}, E_mag={E_mag}")


P=[15.  8.  3.], E=[0.15100671 0.08053691 0.03020134], E_mag=0.17378533390904768
P=[7. 0. 0.], E=[7.44235823e+03 1.72984872e-06 8.34200050e-07], E_mag=7442.358227156817


In [ ]:
#inside case might need to be stronger 